In [1]:
!pip install medmnist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=8d16ade0354dd66c8b0397eed57a06dde883cc37a8536a9829932af90c83dd49
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire


In [2]:
from medmnist import BloodMNIST
import matplotlib.pyplot as plt
from torchvision import models
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.models import VGG16_Weights
import cv2
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [3]:
# Select device to accelerate the computation
#check if CUDA is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#check if MPS is available
device = torch.device("mps" if torch.backends.mps.is_available() else device)

print(device)

cuda:0


In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

train_dataset = BloodMNIST(
    root='/kaggle/input/standardized-biomedical-images-medmnist', split='train', download=False, transform=transform)
val_dataset = BloodMNIST(root='/kaggle/input/standardized-biomedical-images-medmnist', split='val',
                         download=False, transform=transform)
test_dataset = BloodMNIST(root='/kaggle/input/standardized-biomedical-images-medmnist', split='test',
                          download=False, transform=transform)


# Data Loaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
# get sample data
sample, label = train_dataset[0]
print(f"Lable: {label}")
print(f"Image size: {sample.shape}")

Lable: [7]
Image size: torch.Size([3, 224, 224])


In [6]:
def get_vgg16_model():
    # Load pre-trained VGG16 model
    model = models.vgg16(weights=VGG16_Weights.DEFAULT)
    # Remove the existing classification head by replacing it
    model.classifier = nn.Sequential(
        nn.Linear(25088, 4096),  # First fully connected layer (unchanged)
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096, 256),  # Replace second FC layer with smaller one
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(256, 8),  # 8-class classification 
        # nn.Softmax(dim=1)  # Softmax for classification
    )
    return model

In [7]:
def to_rgb_image_np(gray_image_np):
    return np.dstack([gray_image_np, gray_image_np, gray_image_np])


def to_rgb_image_nps(gray_image_nps):
    for i in range(len(gray_image_nps)):
        gray_image_nps[i] = to_rgb_image_np(gray_image_nps[i])
    return gray_image_nps

In [8]:
def train(model, dataloader, optimizer=None, class_criterion=None):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(dataloader, desc="Training batch", leave=False):
        images = images.to(device)
        labels = labels.view(-1).long().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        
        loss = class_criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()* images.size(0)
    return running_loss / len(dataloader.dataset)


def validate(model, dataloader, class_criterion=None):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Validation batch", leave=False):
            images = images.to(device)
            labels = labels.view(-1).long().to(device)

            outputs = model(images)
            loss = class_criterion(outputs, labels)
            running_loss += loss.item()* images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return running_loss / len(dataloader.dataset)

def test(model, dataloader, class_criterion=None):
    model.eval()
    y_true, y_pred = [], []
    running_loss = 0.0
    first_run = True
    show_sample = True

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Testing batch", leave=False):
            images = images.to(device)
            labels = labels.view(-1).long().to(device)

            outputs = model(images)
            loss = class_criterion(outputs, labels) if class_criterion else 0.0
            
            running_loss += loss.item() * labels.size(0)
            
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(outputs.argmax(dim=1).cpu().numpy())

            if show_sample:
                count = 0
                # print the sample of misclassified images
                for i in range(len(labels)):
                    if labels[i] != outputs.argmax(dim=1)[i]:
                        print(
                            f"True: {labels[i].item()}, Predicted: {outputs.argmax(dim=1)[i].item()}")
                        plt.imshow(images[i].cpu().permute(1, 2, 0))
                        plt.show()
                        count += 1
                        if count == 3:
                            show_sample = False
                            break

    # 计算指标
    acc = accuracy_score(y_true, y_pred) * 100
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted', zero_division=0)
    avg_loss = running_loss / len(dataloader.dataset)


    print(f"Accuracy: {acc:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print(f"Average Loss: {avg_loss:.4f}")

    # 结果字典
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'loss': avg_loss
    }

In [ ]:
# Train VGG16
# Use Cross-Entropy Loss and Adam optimizer.
model = get_vgg16_model()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=25,  # maximum number of epochs
    eta_min=1e-5  # minimum learning rate
)
epochs = 25
early_stop = 5
train_loss = []
val_loss = []
min_val_loss = np.inf
early_stop_counter = 0
best_model = None

# use tqdm for progress bar
for epoch in tqdm(range(epochs), desc="Epochs", leave=True):
    # Train for one epoch
    train_loss_epoch = train(
        model, train_loader, optimizer, criterion)
    train_loss.append(train_loss_epoch)

    # Validate on validation set
    val_loss_epoch = validate(model, val_loader, criterion)
    val_loss.append(val_loss_epoch)

    scheduler.step()
    # Early stopping logic
    if val_loss_epoch < min_val_loss:
        min_val_loss = val_loss_epoch
        early_stop_counter = 0
        best_model = model.state_dict()
        # Save the model
    else:
        early_stop_counter += 1

    if early_stop_counter >= early_stop:
        print(f"Early stopping at epoch {epoch}")
        break

# Plot the results
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend()
plt.show()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 192MB/s]  


Epochs:   0%|          | 0/25 [00:00<?, ?it/s]

Training batch:   0%|          | 0/94 [00:00<?, ?it/s]

Validation batch:   0%|          | 0/14 [00:00<?, ?it/s]

Training batch:   0%|          | 0/94 [00:00<?, ?it/s]

...

Training batch:   0%|          | 0/94 [00:00<?, ?it/s]

Validation batch:   0%|          | 0/14 [00:00<?, ?it/s]

Training batch:   0%|          | 0/94 [00:00<?, ?it/s]

Validation batch:   0%|          | 0/14 [00:00<?, ?it/s]

Training batch:   0%|          | 0/94 [00:00<?, ?it/s]

In [ ]:
# test model
model.load_state_dict(best_model)
model.eval()

# calculate accuracy, precision, recall, and F1 score
test(model, test_loader, criterion)

In [ ]:
def Skeletonization(image):
    skel = np.zeros(image.shape, np.uint8) 
    ret, image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY) 
    element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
    done = False

    while not done:
        eroded = cv2.erode(image, element) 
        temp = cv2.dilate(eroded, element) 
        temp = cv2.subtract(image, temp) 
        skel = cv2.bitwise_or(skel, temp) 
        image = eroded.copy()

        if cv2.countNonZero(image) == 0: 
            done = True

    return skel

In [ ]:
first_erosion = True
class ErosionTransform:
    def __init__(self):
        pass

    def __call__(self, image):
        print(f"image type: {image.mode}")
        if first_erosion:
            plt.imshow(image)
            plt.show()
        gray_image = image.convert("L")

        gray_image_np = np.array(gray_image)
        gray_image_np = cv2.erodes(
            gray_image_np, None, iterations=1)

        rgb_image_np = np.dstack(
            [gray_image_np, gray_image_np, gray_image_np])

        rgb_image = Image.fromarray(rgb_image_np)
        if first_erosion:
            plt.imshow(rgb_image)
            plt.show()
            first_erosion = False

        return rgb_image


first_skeletonization = True
class SkeletonizationTransform:
    def __init__(self):
        pass

    def __call__(self, image):
        if first_skeletonization:
            plt.imshow(image)
            plt.show()
        
        gray_image = image.convert("L")

        gray_image_np = np.array(gray_image)
        gray_image_np = Skeletonization(gray_image_np)

        rgb_image_np = np.dstack(
            [gray_image_np, gray_image_np, gray_image_np])

        rgb_image = Image.fromarray(rgb_image_np)
        if first_skeletonization:
            plt.imshow(rgb_image)
            plt.show()
            first_skeletonization = False

        return rgb_image

In [ ]:
trasnform = transforms.Compose([
    ErosionTransform(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

train_dataset = BloodMNIST(
    root='/kaggle/input/standardized-biomedical-images-medmnist', split='train', download=False, transform=transform)
val_dataset = BloodMNIST(root='/kaggle/input/standardized-biomedical-images-medmnist', split='val',
                         download=False, transform=transform)
test_dataset = BloodMNIST(root='/kaggle/input/standardized-biomedical-images-medmnist', split='test',
                          download=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Train VGG16
# Use Cross-Entropy Loss and Adam optimizer.
model = get_vgg16_model()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=25,  # maximum number of epochs
    eta_min=1e-5  # minimum learning rate
)
epochs = 25
early_stop = 5
train_loss = []
val_loss = []
min_val_loss = np.inf
early_stop_counter = 0
best_model = None

# use tqdm for progress bar
for epoch in tqdm(range(epochs), desc="Epochs", leave=True):
    # Train for one epoch
    train_loss_epoch = train(
        model, train_loader, optimizer, criterion)
    train_loss.append(train_loss_epoch)

    # Validate on validation set
    val_loss_epoch = validate(model, val_loader, criterion)
    val_loss.append(val_loss_epoch)
    scheduler.step()

    # Early stopping logic
    if val_loss_epoch < min_val_loss:
        min_val_loss = val_loss_epoch
        early_stop_counter = 0
        best_model = model.state_dict()
    else:
        early_stop_counter += 1

    if early_stop_counter >= early_stop:
        print(f"Early stopping at epoch {epoch}")
        break

# Plot the results
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend()
plt.show()

In [ ]:
# test model
model.load_state_dict(best_model)
model.eval()

# calculate accuracy, precision, recall, and F1 score
test(model, test_loader, criterion)

In [ ]:
trasnform = transforms.Compose([
    SkeletonizationTransform(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

train_dataset = BloodMNIST(
    root='/kaggle/input/standardized-biomedical-images-medmnist', split='train', download=False, transform=transform)
val_dataset = BloodMNIST(root='/kaggle/input/standardized-biomedical-images-medmnist', split='val',
                         download=False, transform=transform)
test_dataset = BloodMNIST(root='/kaggle/input/standardized-biomedical-images-medmnist', split='test',
                          download=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Train VGG16
# Use Cross-Entropy Loss and Adam optimizer.
model = get_vgg16_model()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=25,  # maximum number of epochs
    eta_min=1e-5  # minimum learning rate
)
epochs = 25
early_stop = 5
train_loss = []
val_loss = []
min_val_loss = np.inf
early_stop_counter = 0
best_model = None

# use tqdm for progress bar
for epoch in tqdm(range(epochs), desc="Epochs", leave=True):
    # Train for one epoch
    train_loss_epoch = train(
        model, train_loader, optimizer, criterion)
    train_loss.append(train_loss_epoch)

    # Validate on validation set
    val_loss_epoch = validate(model, val_loader, criterion)
    val_loss.append(val_loss_epoch)
    scheduler.step()
    
    # Early stopping logic
    if val_loss_epoch < min_val_loss:
        min_val_loss = val_loss_epoch
        early_stop_counter = 0
        best_model = model.state_dict()
    else:
        early_stop_counter += 1

    if early_stop_counter >= early_stop:
        print(f"Early stopping at epoch {epoch}")
        break

# Plot the results
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend()
plt.show()

In [ ]:
# test model
model.load_state_dict(best_model)
model.eval()

# calculate accuracy, precision, recall, and F1 score
test(model, test_loader, criterion)